In [26]:
import numpy as np

# r = np.load('Checkpoints/results_EEGNet_4_2.npy')
# r = np.load('Checkpoints/results_EEGNet_8_2.npy')
# r = np.load('Checkpoints/results_CLSTM.npy')
# r = np.load('Checkpoints/results_DC.npy')
# r = np.load('Checkpoints/results_SC.npy')
# r = np.load('Checkpoints/results_SEFFNet.npy')
# r = np.load('Checkpoints4/results_PGCFMTL1.npy')
# r2 = np.load('Checkpoints5/results_PGCFMTL1.npy')
# r3 = np.load('Checkpoints6/results_PGCFMTL1.npy')
# r1 = np.load('Checkpoints/results_PGCFMTL1.npy')
# r = np.load('Checkpoints/results_Min2Net.npy')
# r = np.load('Checkpoints/results_EEGfusion.npy')
# r = np.load('Checkpoints/results_LSTM.npy')
r = np.load('Checkpoints/results_pCNN.npy')
print(r)
# print(r.shape)
# print(r1)
# print(r2)
# print(r3)

[[2.5520e+01 2.5600e-01 5.0000e-01 7.0000e-03]
 [2.5690e+01 2.5500e-01 5.1600e-01 9.0000e-03]
 [2.6740e+01 2.6700e-01 5.0400e-01 2.3000e-02]
 [2.6560e+01 2.6400e-01 5.0200e-01 2.1000e-02]
 [2.5170e+01 2.5200e-01 5.0900e-01 2.0000e-03]
 [2.5936e+01 2.5900e-01 5.0600e-01 1.2000e-02]]


In [27]:
print(np.std(r[:5, 0]))
print(np.std(r[:5, 1]))
print(np.std(r[:5, 2]))
print(np.std(r[:5, 3]))

0.6092815441156892
0.005706137047074847
0.005741080037762934
0.008187795796183488


In [11]:
print(r[:5, 0])

[35.07 48.   56.51 28.73 42.1 ]


In [8]:
r[2][0]

56.51

In [9]:
from tensorflow.keras.models import load_model   
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score 
from utils import *
import mne

results = np.zeros((6, 4))
random_seeds = [6, 16, 66, 88, 166]
save_path = '/home/henrywang/testEEGModels/BCIIV2a/data/'


for i in range(5):
    _, _, subjs_te = split_datasets(seed=random_seeds[i])
    data_X_te = [mne.filter.resample(np.load(save_path+'S{0}_X.npy'.format(i)), down = 1.5625, npad='auto') for i in subjs_te]
    data_teX_c = np.vstack(data_X_te)
    data_y_te = [onehot_labels(np.load(save_path+'S{0}_y.npy'.format(i)).reshape(-1, 1)) for i in subjs_te]
    data_teY_c = np.vstack(data_y_te)
    data_teX_c = std(data_teX_c)
    data_teX_c = np.expand_dims(data_teX_c, axis = -1)
    model = load_model('Checkpoints/EEGNet_8_2-{0}.hdf5'.format(i+1))  
    y_pred_prob = model.predict(data_teX_c)
    y_pred = np.argmax(y_pred_prob, axis = 1)
    y_true = np.argmax(data_teY_c, axis = 1)
    acc = 100 * np.mean(y_true  == y_pred)
    f1 = f1_score(y_true, y_pred, average='macro') 
    auc = roc_auc_score(y_true, y_pred_prob, multi_class ='ovr', average='macro')
    kappa = cohen_kappa_score(y_true, y_pred) 
    results[i, 0] = round(acc, 2)
    results[i, 1] = round(f1, 3)
    results[i, 2] = round(auc, 3)
    results[i, 3] = round(kappa, 3)
results[5, :] = np.round(np.mean(results[:-1], axis = 0), decimals = 3)    

In [10]:
np.save('Checkpoints/results_EEGNet_8_2.npy', results)

In [11]:
results

array([[49.22 ,  0.489,  0.744,  0.323],
       [46.27 ,  0.463,  0.714,  0.284],
       [42.97 ,  0.424,  0.702,  0.24 ],
       [48.96 ,  0.489,  0.73 ,  0.319],
       [36.89 ,  0.36 ,  0.647,  0.159],
       [44.862,  0.445,  0.707,  0.265]])

In [12]:
print(np.std(results[:5, 0]))
print(np.std(results[:5, 1]))
print(np.std(results[:5, 2]))
print(np.std(results[:5, 3]))

4.5804078421031456
0.04870728898224577
0.03338023367204009
0.060830913193868796


In [18]:
from tensorflow.keras.models import load_model   
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score 
from utils import *
import mne


class gatemodule(Layer):

    def __init__(self, features_shape=48, **kwargs):
        super(gatemodule, self).__init__()
        self.shape = features_shape

    def build(self, input_shape):


        self.W1_f = self.add_weight(shape=(self.shape*2, self.shape),
                                   initializer='glorot_uniform',
                                   trainable=True,
                                   name='W1_f')
        
        self.b1_f = self.add_weight(shape=(self.shape,),
                                        initializer='zeros',
                                        trainable=True,
                                        name='b1_f')
        self.W2_f = self.add_weight(shape=(self.shape*2, self.shape),
                                   initializer='glorot_uniform',
                                   trainable=True,
                                   name='W2_f')
        
        self.b2_f = self.add_weight(shape=(self.shape,),
                                        initializer='zeros',
                                        trainable=True,
                                        name='b2_f')
        self.W3_f = self.add_weight(shape=(self.shape*2, self.shape),
                                   initializer='glorot_uniform',
                                   trainable=True,
                                   name='W3_f')
        
        self.b3_f = self.add_weight(shape=(self.shape,),
                                        initializer='zeros',
                                        trainable=True,
                                        name='b3_f')


    def call(self, inputs):
        input1, input2 = inputs
        concat = tf.concat([input1, input2], axis=-1)
        G1 = tf.nn.sigmoid(tf.matmul(concat, self.W1_f) + self.b1_f)
        G2 = tf.nn.sigmoid(tf.matmul(concat, self.W2_f) + self.b2_f)
        G3 = tf.nn.sigmoid(tf.matmul(concat, self.W3_f) + self.b3_f)
        Gated_out1 = G1*input1+G2*input2
        Gated_out2 = (1-G3)*input1+input2


        return Gated_out1, Gated_out2

def mse_segmentation_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

results = np.zeros((6, 4))
random_seeds = [6, 16, 66, 88, 166]
save_path = '/home/henrywang/testEEGModels/BCIIV2a/data/'


for i in range(5):
    _, _, subjs_te = split_datasets(seed=random_seeds[i])
    data_X_te = [mne.filter.resample(np.load(save_path+'S{0}_X.npy'.format(i)), down = 1.171825, npad='auto') for i in subjs_te]
    data_teX_c = np.vstack(data_X_te)
    data_y_te = [onehot_labels(np.load(save_path+'S{0}_y.npy'.format(i)).reshape(-1, 1)) for i in subjs_te]
    data_teY_c = np.vstack(data_y_te)
    data_teX_c = std(data_teX_c)
    data_teX_c = np.swapaxes(data_teX_c, 1, 2)
    data_teX_c = np.expand_dims(data_teX_c, axis = -1)
    model = load_model('Checkpoints/PGCFMTL-{0}.hdf5'.format(i+1), custom_objects={'gatemodule': gatemodule, 'mse_segmentation_loss': mse_segmentation_loss})  
    _, y_pred_prob = model.predict([data_teX_c, data_teX_c])
    y_pred = np.argmax(y_pred_prob, axis = 1)
    y_true = np.argmax(data_teY_c, axis = 1)
    acc = 100 * np.mean(y_true  == y_pred)
    f1 = f1_score(y_true, y_pred, average='macro') 
    auc = roc_auc_score(y_true, y_pred_prob, multi_class ='ovr', average='macro')
    kappa = cohen_kappa_score(y_true, y_pred) 
    results[i, 0] = round(acc, 2)
    results[i, 1] = round(f1, 3)
    results[i, 2] = round(auc, 3)
    results[i, 3] = round(kappa, 3)
results[5, :] = np.round(np.mean(results[:-1], axis = 0), decimals = 3)

ValueError: multi_class must be in ('ovo', 'ovr')

: 

In [12]:
1224/1125

1.088

In [13]:
1224/8

153.0